<a href="https://colab.research.google.com/github/xuxiufeng/Spam_ham_Classification/blob/main/Naive%20Bayes%20and%20Logistic%20Regression%20for%20Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!unzip enron1_test.zip
!unzip enron1_train.zip

In [2]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
              "you're", "you've","you'll", "you'd", 'your', 'yours', 'yourself', 
              'yourselves', 'he', 'him', 'his','himself', 'she', "she's", 'her', 
              'hers', 'herself', 'it', "it's", 'its', 'itself','they', 'them', 
              'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
              'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
              'was', 'were', 'be','been', 'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a','an', 'the', 'and', 'but', 'if', 
              'or', 'because', 'as', 'until', 'while', 'of', 'at','by', 'for', 
              'with', 'about', 'against', 'between', 'into', 'through', 'during',
              'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 
              'out', 'on','off', 'over', 'under', 'again', 'further', 'then', 'once', 
              'here', 'there', 'when','where', 'why', 'how', 'all', 'any', 'both', 
              'each', 'few', 'more', 'most', 'other','some', 'such', 'no', 'nor', 'not', 
              'only', 'own', 'same', 'so', 'than', 'too', 'very','s', 't', 'can', 'will', 
              'just', 'don', "don't", 'should', "should've", 'now', 'd','ll', 'm', 'o', 're', 
              've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 
              'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
              'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",
              'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
              "won't", 'wouldn', "wouldn't"]

In [3]:
import copy
import glob
import os
import re
from collections import Counter
import codecs

## Bag of Words model

In [4]:
def import_train_data(path, train_data_type):
    """
    This function takes the dataset name and returns the files for the spam and ham text files for the train data
    :param train_data_type: In this if it is true then we have train data else test data
    :param data_set_name: This is the dataset name
    :return: We return two lists in which consists of the spam and ham for given dataset
    """
    files_ham = []
    files_spam = []
    total_data = ""
    if train_data_type == True:
        path = path + '/train'
    else:
        path = path + '/test'
    path_ham = path + '/ham/'
    path_spam = path + '/spam/'
    files_list_spam = os.listdir(path_spam)
    files_list_ham = os.listdir(path_ham)
    for spam_files in files_list_spam:
      path_all_spam = path_spam + spam_files
      files_spam.append(open(path_all_spam, "r", errors = 'ignore').read())
      total_data = total_data + " " + open(path_all_spam, "r", errors = 'ignore').read()
    for ham_files in files_list_ham:
      path_all_ham = path_ham + ham_files
      files_ham.append(open(path_all_ham, "r", errors = 'ignore').read())
      total_data = total_data + " " + open(path_all_ham, "r", errors = 'ignore').read()
    # we find the size of the dataset and the number of instances with spam and number of instances with ham
    size_of_total_dataset = len(files_list_ham) + len(files_list_spam)
    size_of_ham_dataset = len(files_list_ham)
    size_of_spam_dataset = len(files_list_spam)
    return files_spam, files_ham, total_data, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset

In [ ]:
import_train_data('/content/enron1', True)

In [6]:
def convert_to_bag_of_words(dataset_name, train_data_type):
    """
    This function returns the bag of words for given dataset
    :param dataset_name: This is the dataset name
    :param train_data_type: In this if it is true then we have train data else test data
    :return: We return the bag for words representation for spam and ham files
    """
    spam_file, ham_file, total_data, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset = import_train_data(
        dataset_name, train_data_type)
    total_file_dictionary = {}
    total_file_data = re.findall("[a-zA-Z]+", total_data)
    text_in_all_document = {}
    # at first we find all the words in the given dataset and find the frequencies in the whole dataset
    for each_word in total_file_data:
        each_word = each_word.lower()
        if each_word in total_file_dictionary:
            continue
        else:
            # I got the list of stem words from the nltk library
            if each_word not in stop_words:
                total_file_dictionary[each_word] = 0
        if each_word in text_in_all_document:
            if each_word not in stop_words:
                text_in_all_document[each_word] = text_in_all_document[each_word] + 1
        else:
            if each_word not in stop_words:
                text_in_all_document[each_word] = 1
    # In the following steps we find the words in the spam dataset and create the bag of words
    spam_email_bag_of_words = []
    spam_mail_in_all_documents = {}
    # The frequencies of each words are stored in the case of bag of words.
    for each_spam_mail in spam_file:
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_spam_mail1 = re.findall("[a-zA-Z]+", each_spam_mail)
        # Here we create the bag of words for each document and append it in a list
        for each_word in each_spam_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = temp_dict[each_word] + 1
        # Here we store all the words in the spam dataset
        spam_mail_in_all_documents = Counter(spam_mail_in_all_documents) + Counter(temp_dict)
        spam_email_bag_of_words.append(temp_dict)
    # In the following steps we find all the ham words and add them in the bag of words
    ham_mail_in_all_documents = {}
    ham_email_bag_of_words = []
    for each_ham_mail in ham_file:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_ham_mail1 = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in each_ham_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = temp_dict[each_word] + 1
        # Here we store all the words in the ham dataset
        ham_mail_in_all_documents = Counter(ham_mail_in_all_documents) + Counter(temp_dict)
        ham_email_bag_of_words.append(temp_dict)
    return spam_email_bag_of_words, ham_email_bag_of_words, text_in_all_document, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary

In [7]:
spam_email_bag_of_words, ham_email_bag_of_words, text_in_all_document, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary = convert_to_bag_of_words('/content/enron1', True)

In [ ]:
spam_email_bag_of_words

## Bernoulli model

In [13]:
def convert_to_bernoulli_model(dataset_name, train_data_type):
    """
    This function returns the bernoulli model for given dataset
    :param dataset_name: This is the dataset name
    :param train_data_type: In this if it is true then we have train data else test data
    :return: We return the bernoulli model representation for spam and ham files
    """
    spam_file, ham_file, total_data, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset = import_train_data(
        dataset_name, train_data_type)
    total_file_dictionary = {}
    total_file_data = re.findall("[a-zA-Z]+", total_data)
    # at first we find all the words in the given dataset and find the occurrences in the whole dataset
    for each_word in total_file_data:
        # The words are converted to their lower case forms
        each_word = each_word.lower()
        if each_word in total_file_dictionary:
            continue
        else:
            if each_word not in stop_words:
                total_file_dictionary[each_word] = 0
    # In the following steps we find the words in the spam dataset and create the model
    spam_email_bernoulli_model = []
    spam_mail_in_all_documents = {}
    for each_spam_mail in spam_file:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_spam_mail1 = re.findall("[a-zA-Z]+", each_spam_mail)
        for each_word in each_spam_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = 1
                # Here we store all the words in the ham dataset
                spam_mail_in_all_documents[each_word] = 1
        temp_list = list(temp_dict.values())
        spam_email_bernoulli_model.append(temp_dict)
    # In the following steps we find the words in the ham dataset and create the model
    ham_email_bernoulli_model = []
    ham_mail_in_all_documents = {}
    for each_ham_mail in ham_file:
        # Here we create the bag of words for each document and append it in a list
        temp_dict = copy.deepcopy(total_file_dictionary)
        each_ham_mail1 = re.findall("[a-zA-Z]+", each_ham_mail)
        for each_word in each_ham_mail1:
            each_word = each_word.lower()
            if each_word in temp_dict:
                temp_dict[each_word] = 1
                ham_mail_in_all_documents[each_word] = 1
        ham_email_bernoulli_model.append(temp_dict)
    return spam_email_bernoulli_model, ham_email_bernoulli_model, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary

In [14]:
spam_email_bernoulli_model, ham_email_bernoulli_model, spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_total_dataset, size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary= convert_to_bernoulli_model('/content/enron1', True)

In [ ]:
from decimal import Decimal
from math import log10 as log


def train_multinomial_NB(spam_email_bag_of_words, ham_email_bag_of_words, text_in_all_document,
                         spam_mail_in_all_documents, ham_mail_in_all_documents, size_of_total_dataset,
                         size_of_spam_dataset, size_of_ham_dataset, total_file_dictionary):
    """
    This is the main algorithm to train the multinomial Naive Bayes
    :param total_file_dictionary: This is the list containing all the words in the training examples
    :param spam_email_bag_of_words:  This is the list of all words in each spam document (1st value is for first document and so on)
    :param ham_email_bag_of_words: This is the list of all words in each ham document (1st value is for first document and so on)
    :param text_in_all_document: This is the total text in all documents with their frequencies
    :param spam_mail_in_all_documents: This is the total text in all spam documents with their frequencies
    :param ham_mail_in_all_documents: This is the total text in all ham documents with their frequencies
    :param size_of_total_dataset: This is total number of files in all dataset
    :param size_of_spam_dataset: This is total number of files in all spam dataset
    :param size_of_ham_dataset: This is total number of files in all ham dataset
    :return: prior and conditional probability for both spam and ham(all these values are in log)
    """
    no_of_docs = size_of_total_dataset
    # We will first do it for the spam
    no_of_spam_docs = size_of_spam_dataset
    # We create the variables to store the values
    prior = {}
    conditional_probability = {}
    conditional_probability["spam"] = {}
    conditional_probability["ham"] = {}
    conditional_probability_of_non_occurring_word = {}
    conditional_probability_of_non_occurring_word["spam"] = {}
    conditional_probability_of_non_occurring_word["ham"] = {}
    value = Decimal(no_of_spam_docs / float(no_of_docs))
    # First we calculate the priors for the spam and ham dataset
    prior["spam"] = log(value)
    no_of_ham_docs = size_of_ham_dataset
    total_number_of_words_in_ham = sum(ham_mail_in_all_documents.itervalues())
    prior["ham"] = log(no_of_ham_docs / float(no_of_docs))
    total_number_of_words_in_spam = sum(spam_mail_in_all_documents.itervalues())
    # Now we calculate the values for the conditional probabilities
    for each_word in list(spam_mail_in_all_documents):
        conditional_probability["spam"][each_word] = log((spam_mail_in_all_documents[each_word] + 1) / (
            float(total_number_of_words_in_spam + len(text_in_all_document))))

    # Now we will do the same procedure for ham docs
    for each_word in list(ham_mail_in_all_documents):
        conditional_probability["ham"][each_word] = log((ham_mail_in_all_documents[each_word] + 1) / (
            float(total_number_of_words_in_ham + len(text_in_all_document))))
    # These are the values for the conditional probabilities whose words are not in the training dataset
    conditional_probability_of_non_occurring_word["ham"] = log(
        1 / (float(total_number_of_words_in_ham + len(text_in_all_document))))
    conditional_probability_of_non_occurring_word["spam"] = log(
        1 / (float(total_number_of_words_in_spam + len(text_in_all_document))))
    return prior, conditional_probability, conditional_probability_of_non_occurring_word


def test_multinomial_naive_bayes(prior, conditional_probability, conditional_probability_of_non_occurring_word,
                                 an_email_bag_of_words_test):
    """
    :param conditional_probability_of_non_occurring_word: This is the conditional probability for each word in the testing set which is not in the training data
    :param prior: This is the prior for all classes
    :param conditional_probability:  This is the conditional probability for each word in vocabulary in spam and ham data
    :param an_email_bag_of_words_test: This is the given test instance we want to classify
    :return: the class of the given email
    """
    score = {}
    for each_class in list(prior):
        score[each_class] = prior[each_class]
        for each_word in list(an_email_bag_of_words_test):
            if an_email_bag_of_words_test[each_word] != 0:
                try:
                    score[each_class] += conditional_probability[each_class][each_word]
                # This is the case if the word was not in the train data and thus the laplace pruning gives this result
                except KeyError:
                    score[each_class] += conditional_probability_of_non_occurring_word[each_class]
    # Here we are taking spam as 1 and ham as -1
    if score["spam"] > score["ham"]:
        return 1
    else:
        return 0